In [3]:
!pip install transformers==4.44.2 joblib==1.4.2 scikit-learn==1.6.0 numpy==1.26.4 pandas==2.2.3 scipy==1.13.1 seaborn==0.13.2 tqdm==4.66.5 lightgbm==4.5.0 xgboost==2.1.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 67.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 86.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 45.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 11.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 80.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    U

In [4]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [5]:
# Load datasets
train_df = pd.read_csv('/kaggle/input/mdck-set/Train_MDCK.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/mdck-set/Test_MDCK.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [6]:
tokenizer = AutoTokenizer.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
model = AutoModelForSequenceClassification.from_pretrained('seyonec/ChemBERTa-zinc-base-v1', num_labels=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Custom dataset class
class SMILESDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=325):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        smiles = self.dataframe.iloc[idx]['SMILES']
        permeability = self.dataframe.iloc[idx]['Permeability']
        inputs = self.tokenizer(smiles, return_tensors='pt', padding="max_length", truncation=True, max_length=self.max_length)
        
        input_ids = inputs['input_ids'].squeeze(0)  # Shape: (sequence_length,)
        attention_mask = inputs['attention_mask'].squeeze(0)  # Shape: (sequence_length,)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(permeability, dtype=torch.float)
        }


# datasets
train_dataset = SMILESDataset(train_df, tokenizer)
test_dataset = SMILESDataset(test_df, tokenizer)
batch_size = 16
# data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/179M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/ChemBERTa-zinc-base-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 20

In [8]:
# Training loop
from tqdm import tqdm
for epoch in range(num_epochs):
    print(f"Entered Epoch {epoch + 1}")
    model.train()
    train_loss = 0

    for batch in tqdm(train_loader, desc=f'Training Epoch {epoch + 1}/{num_epochs}', unit='batch'):
        optimizer.zero_grad()

        # Move all batch tensors to device
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch["labels"].unsqueeze(1)  # still shape: (batch_size, 1)

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels
        )
        loss = outputs.loss
        train_loss += loss.item()

        # Backprop and optimizer step
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}')

Entered Epoch 1


Training Epoch 1/20: 100%|██████████| 4/4 [00:02<00:00,  1.89batch/s]


Epoch 1/20 - Train Loss: 19.0339
Entered Epoch 2


Training Epoch 2/20: 100%|██████████| 4/4 [00:01<00:00,  3.01batch/s]


Epoch 2/20 - Train Loss: 3.7463
Entered Epoch 3


...

Epoch 18/20 - Train Loss: 0.5215
Entered Epoch 19


Training Epoch 19/20: 100%|██████████| 4/4 [00:01<00:00,  2.84batch/s]


Epoch 19/20 - Train Loss: 0.2954
Entered Epoch 20


Training Epoch 20/20: 100%|██████████| 4/4 [00:01<00:00,  2.83batch/s]

Epoch 20/20 - Train Loss: 0.5060


In [9]:
model_name = 'ChemBERTa_model_1_mdck'
model_save_path = f'/kaggle/working/{model_name}'
os.makedirs(model_save_path, exist_ok=True)

tokenizer.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)

print(f'Model and tokenizer saved to {model_save_path}')

Model and tokenizer saved to /kaggle/working/ChemBERTa_model_1_mdck


In [10]:
from scipy.stats import pearsonr, spearmanr

model.eval()
test_loss = 0
test_true_labels = []
predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing', unit='batch'):
      
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].unsqueeze(1).to(device).float()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        test_loss += loss.item()

        test_true_labels.extend(labels.cpu().numpy())
        preds = outputs.logits.squeeze().cpu().numpy()  
        predictions.extend(preds)

# Final test loss
avg_test_loss = test_loss / len(test_loader)
print(f'Test Loss: {avg_test_loss:.4f}')

test_true_labels = np.array(test_true_labels).flatten()
predictions = np.array(predictions)
print(test_true_labels.shape)
print(predictions.shape)

# Performance metrics
mse = mean_squared_error(test_true_labels, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_true_labels, predictions)
r2 = r2_score(test_true_labels, predictions)
PCC,_ = pearsonr(test_true_labels, predictions)
SCC,_ = spearmanr(test_true_labels, predictions)
# Print performance metrics
print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')
print(f'R^2 Score: {r2:.4f}')
print(f'Pearson Correlation Coefficient: {PCC:.4f}')
print(f'Spearman Correlation Coefficient: {SCC:.4f}')

# Print hyperparameters
print("Hyperparameters:")
print(f"Learning Rate: {5e-5}")
print(f"Batch Size: 16")
print(f"Epochs: {num_epochs}")

Testing: 100%|██████████| 1/1 [00:00<00:00,  7.87batch/s]

Test Loss: 0.5383
(13,)
(13,)
Mean Squared Error: 0.5383
Root Mean Squared Error: 0.7337
Mean Absolute Error: 0.5965
R^2 Score: 0.0849
Pearson Correlation Coefficient: 0.5554
Spearman Correlation Coefficient: 0.3824
Hyperparameters:
Learning Rate: 5e-05
Batch Size: 16
Epochs: 20


In [11]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model_name = 'ChemBERTa_model_1_mdck'
model_save_path = f'/kaggle/working/{model_name}'

if not os.path.exists(model_save_path):
    raise FileNotFoundError(f"The model directory {model_save_path} does not exist.")

tokenizer = AutoTokenizer.from_pretrained(model_save_path)
model = AutoModel.from_pretrained(model_save_path).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at /kaggle/working/ChemBERTa_model_1_mdck and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Load your datasets
train_df = pd.read_csv('/kaggle/input/mdck-set/Train_MDCK.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/mdck-set/Test_MDCK.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [13]:
train_encodings = tokenizer(list(train_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")
test_encodings = tokenizer(list(test_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")

In [14]:
from tqdm import tqdm 
batch_size = 16 

def generate_embeddings(encodings, batch_size):
    embeddings = []
    model.eval() 
    with torch.no_grad():
        for i in tqdm(range(0, len(encodings['input_ids']), batch_size), desc="Processing batches"):
            batch = {key: val[i:i + batch_size].to(device) for key, val in encodings.items()}  
            outputs = model(**batch)
            embeddings.append(outputs.last_hidden_state)
    return torch.cat(embeddings, dim=0)

In [15]:
train_embeddings = generate_embeddings(train_encodings, batch_size)
print(train_embeddings.shape)
train_embeddings = torch.mean(train_embeddings, dim=1)
print(train_embeddings.shape)

Processing batches: 100%|██████████| 4/4 [00:00<00:00, 20.17it/s]


torch.Size([51, 151, 768])
torch.Size([51, 768])


In [16]:
column_names = [f'x_fine_emb_ChemBerta{i}' for i in range(train_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=train_embeddings.cpu().numpy(), columns=column_names)
train_data = pd.concat([train_df, embeddings_df], axis=1)

In [17]:
test_embeddings = generate_embeddings(test_encodings, batch_size)
print(test_embeddings.shape)
test_embeddings = torch.mean(test_embeddings, dim=1)
print(test_embeddings.shape)

Processing batches: 100%|██████████| 1/1 [00:00<00:00, 125.71it/s]

torch.Size([13, 157, 768])
torch.Size([13, 768])


In [18]:
column_names = [f'x_fine_emb_ChemBerta{i}' for i in range(test_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=test_embeddings.cpu().numpy(), columns=column_names)
test_data = pd.concat([test_df, embeddings_df], axis=1)

In [19]:
train_data.to_csv("/kaggle/working/Train_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_mdck.csv",index=False)
test_data.to_csv("/kaggle/working/Test_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_mdck.csv",index=False)

In [20]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [21]:
train_data = pd.read_csv("/kaggle/working/Train_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_mdck.csv")
test_data = pd.read_csv("/kaggle/working/Test_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_mdck.csv")

In [22]:
def train_and_test_predict(models, X_train, y_train, X_test, y_test):
    kf = KFold(n_splits=5, shuffle=True, random_state=101)
    results = {}
    predictions = []  

    for model in models:
        model_name = model.__class__.__name__
        predictions_train = []
        actual_y_train = []

        test_predictions_folds = []

        

        for train_index, val_index in kf.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model.fit(X_train_fold, y_train_fold)

            y_pred_fold = model.predict(X_val_fold)
            y_pred_fold = np.clip(y_pred_fold, -10, -3.9)
            predictions_train.extend(y_pred_fold)
            actual_y_train.extend(y_val_fold)

            predictions_test_fold = model.predict(X_test)
            predictions_test_fold = np.clip(predictions_test_fold, -10, -3.9)
            test_predictions_folds.append(predictions_test_fold)


        mse_train = mean_squared_error(actual_y_train, predictions_train)
        mae_train = mean_absolute_error(actual_y_train, predictions_train)
        rmse_train = np.sqrt(mse_train)
        r2_train = r2_score(actual_y_train, predictions_train)
        pearson_train, _ = pearsonr(actual_y_train, predictions_train)
        spearman_train, _ = spearmanr(actual_y_train, predictions_train)


        predictions_test_mean = np.mean(test_predictions_folds, axis=0)
        predictions_test_std = np.std(test_predictions_folds, axis=0)

        mse_test = mean_squared_error(y_test, predictions_test_mean)
        mae_test = mean_absolute_error(y_test, predictions_test_mean)
        rmse_test = np.sqrt(mse_test)
        r2_test = r2_score(y_test, predictions_test_mean)
        print(r2_test)
        pearson_test, _ = pearsonr(y_test, predictions_test_mean)
        spearman_test, _ = spearmanr(y_test, predictions_test_mean)
        
        

        predictions.append({
            'Model': model_name,
            'Y Train pred': predictions_train,
            'Y Test actual': y_test,
            'Test prediction folds': test_predictions_folds,
            'Test Predictions Mean': predictions_test_mean,
            'Test Predictions Std': predictions_test_std,

        })

        results[model_name] = {
            'Train MSE (5 fold cv)': f"{mse_train:.4f}",
            'Train MAE (5 fold cv)': f"{mae_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train R2 (5 fold cv)': f"{r2_train:.4f}",
            'Train PCC (5 fold cv)': f"{pearson_train:.4f}",
            'Train SCC (5 fold cv)': f"{spearman_train:.4f}",
            'Test MSE': f"{mse_test:.4f}",
            'Test MAE': f"{mae_test:.4f}",
            'Test RMSE': f"{rmse_test:.4f}",
            'Test R2': f"{r2_test:.4f}",
            'Test Pearson Correlation': f"{pearson_test:.4f}",
            'Test Spearman Correlation': f"{spearman_test:.4f}",
        }

    results_df = pd.DataFrame(results).T
    predictions_df = pd.DataFrame(predictions)

    return results_df, predictions_df



In [23]:
X_train = train_data.drop(['ID','SMILES','Permeability'],axis=1)
y_train = train_data['Permeability']
print("X_train shape: ",X_train.shape)
print("y_train shape: ",y_train.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

X_test = test_data.drop(['ID','SMILES','Permeability'],axis=1)
y_test = test_data['Permeability']
print("X_test shape: ",X_test.shape)
print("y_test shape: ",y_test.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
models = [
    lgb.LGBMRegressor(objective='regression',metric='rmse',boosting_type='gbdt',num_leaves=31,learning_rate=0.05,random_state=101),
    DecisionTreeRegressor(random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    LinearRegression(), 
    KNeighborsRegressor(n_neighbors=3),
    SVR(),  
    MLPRegressor(random_state=101)
]
result_df, prediction_df = train_and_test_predict(models, X_train,y_train, X_test,  y_test)
result_df

X_train shape:  (51, 768)
y_train shape:  (51,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X_test shape:  (13, 768)
y_test shape:  (13,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000937 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2490
[LightGBM] [Info] Number of data points in the train set: 40, number of used features: 166
[LightGBM] [Info] Start training from score -5.753098
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.5171,0.5899,0.7191,0.2006,0.4706,0.4637,0.5170,0.6282,0.7190,0.1211,0.4435,0.4127
DecisionTreeRegressor,0.7205,0.6168,0.8488,-0.1137,0.4481,0.4537,0.5667,0.6606,0.7528,0.0366,0.4574,0.3471
RandomForestRegressor,0.5110,0.5717,0.7148,0.2101,0.4740,0.5161,0.4688,0.5945,0.6847,0.2030,0.5229,0.4374
GradientBoostingRegressor,0.6616,0.6240,0.8134,-0.0227,0.3542,0.3563,0.4187,0.5545,0.6470,0.2883,0.5879,0.4264
AdaBoostRegressor,0.6156,0.6426,0.7846,0.0483,0.3660,0.3823,0.4878,0.6087,0.6984,0.1708,0.5538,0.4209
XGBRegressor,0.6124,0.6143,0.7825,0.0534,0.4189,0.4336,0.5025,0.5875,0.7089,0.1457,0.5517,0.4319
ExtraTreesRegressor,0.5457,0.5878,0.7387,0.1565,0.4488,0.4196,0.4586,0.5729,0.6772,0.2204,0.5984,0.4209
LinearRegression,5.0916,1.6532,2.2565,-6.8708,-0.0433,0.0163,2.4853,1.3465,1.5765,-3.2250,0.2458,0.0248
KNeighborsRegressor,0.6412,0.6220,0.8007,0.0089,0.3256,0.3644,0.5822,0.6814,0.7630,0.0102,0.3936,0.3283
SVR,0.6208,0.6227,0.7879,0.0403,0.3086,0.3853,0.5178,0.6335,0.7196,0.1197,0.4303,0.3989


In [24]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.5171,0.5899,0.7191,0.2006,0.4706,0.4637,0.5170,0.6282,0.7190,0.1211,0.4435,0.4127
DecisionTreeRegressor,0.7205,0.6168,0.8488,-0.1137,0.4481,0.4537,0.5667,0.6606,0.7528,0.0366,0.4574,0.3471
RandomForestRegressor,0.5110,0.5717,0.7148,0.2101,0.4740,0.5161,0.4688,0.5945,0.6847,0.2030,0.5229,0.4374
GradientBoostingRegressor,0.6616,0.6240,0.8134,-0.0227,0.3542,0.3563,0.4187,0.5545,0.6470,0.2883,0.5879,0.4264
AdaBoostRegressor,0.6156,0.6426,0.7846,0.0483,0.3660,0.3823,0.4878,0.6087,0.6984,0.1708,0.5538,0.4209
XGBRegressor,0.6124,0.6143,0.7825,0.0534,0.4189,0.4336,0.5025,0.5875,0.7089,0.1457,0.5517,0.4319
ExtraTreesRegressor,0.5457,0.5878,0.7387,0.1565,0.4488,0.4196,0.4586,0.5729,0.6772,0.2204,0.5984,0.4209
LinearRegression,5.0916,1.6532,2.2565,-6.8708,-0.0433,0.0163,2.4853,1.3465,1.5765,-3.2250,0.2458,0.0248
KNeighborsRegressor,0.6412,0.6220,0.8007,0.0089,0.3256,0.3644,0.5822,0.6814,0.7630,0.0102,0.3936,0.3283
SVR,0.6208,0.6227,0.7879,0.0403,0.3086,0.3853,0.5178,0.6335,0.7196,0.1197,0.4303,0.3989


In [25]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-5.354418349914161, -5.316879002745812, -6.56...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.131535090400979, -5.469807473519724, -5.3...","[-5.202178993453439, -5.232634570165066, -5.58...","[0.1242382138879358, 0.14373337516374318, 0.15..."
1,DecisionTreeRegressor,"[-4.73, -5.06, -6.85, -5.793469425, -6.9299170...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-6.25, -6.22, -6.17, -5.303671307, -6.569578...","[-5.4937051858, -6.072, -6.15, -5.7197026134, ...","[0.7361113112697358, 0.2060485379710323, 0.601..."
2,RandomForestRegressor,"[-5.48232826188, -5.391520198169998, -6.364656...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.497930698200004, -5.51785827859, -5.81246...","[-5.385590163172003, -5.476410728076001, -5.76...","[0.2006143404347011, 0.10319695746002353, 0.07..."
3,GradientBoostingRegressor,"[-5.667756811353129, -5.54820047151307, -6.851...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.447365744269249, -5.5911185425796, -6.008...","[-5.433312485239279, -5.4738550888872, -5.8895...","[0.39621646412331, 0.25091914429132384, 0.1917..."
4,AdaBoostRegressor,"[-5.30148539195238, -5.30148539195238, -6.1139...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.270455309699999, -5.418806133230769, -5.5...","[-5.256275101524894, -5.47217058383661, -5.522...","[0.1441962744596021, 0.29031927695059795, 0.22..."
5,XGBRegressor,"[-5.764666, -5.147919, -6.4477324, -5.8681383,...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.522504, -5.68638, -5.780648, -5.4122796, ...","[-5.2166243, -5.451589, -5.8832235, -5.617455,...","[0.4216381, 0.30115765, 0.30467716, 0.21026993..."
6,ExtraTreesRegressor,"[-5.373298738399999, -5.228926914429999, -6.07...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.235171998490006, -5.421000258870004, -5.7...","[-5.311246406306003, -5.562271749428002, -5.69...","[0.3257166882703574, 0.20303047231722762, 0.07..."
7,LinearRegression,"[-10.0, -5.74848892264249, -5.871378872182627,...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-8.89543216109687, -10.0, -3.9, -4.253783602...","[-9.187603680712055, -7.499291887549445, -5.67...","[0.8598541725240082, 2.055134308127762, 1.6447..."
8,KNeighborsRegressor,"[-5.613333333333333, -5.786666666666666, -6.07...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.359999999999999, -5.359999999999999, -5.6...","[-5.306666666666667, -5.526000000000001, -5.39...","[0.29388584329444783, 0.13724916514621652, 0.1..."
9,SVR,"[-5.411402691197683, -5.305429792103938, -5.86...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.105315830130418, -5.440219986114503, -5.6...","[-5.217731689524889, -5.38739966704816, -5.591...","[0.3181027661947282, 0.21389718024989055, 0.03..."


In [26]:
result_df.to_csv('/kaggle/working/Results_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_mdck.csv')
prediction_df.to_csv('/kaggle/working/Prediction_data_ChemBERTa-zinc-base-v1_fine_tuned_embeddings_mdck.csv')

In [27]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.5171,0.5899,0.7191,0.2006,0.4706,0.4637,0.5170,0.6282,0.7190,0.1211,0.4435,0.4127
DecisionTreeRegressor,0.7205,0.6168,0.8488,-0.1137,0.4481,0.4537,0.5667,0.6606,0.7528,0.0366,0.4574,0.3471
RandomForestRegressor,0.5110,0.5717,0.7148,0.2101,0.4740,0.5161,0.4688,0.5945,0.6847,0.2030,0.5229,0.4374
GradientBoostingRegressor,0.6616,0.6240,0.8134,-0.0227,0.3542,0.3563,0.4187,0.5545,0.6470,0.2883,0.5879,0.4264
AdaBoostRegressor,0.6156,0.6426,0.7846,0.0483,0.3660,0.3823,0.4878,0.6087,0.6984,0.1708,0.5538,0.4209
XGBRegressor,0.6124,0.6143,0.7825,0.0534,0.4189,0.4336,0.5025,0.5875,0.7089,0.1457,0.5517,0.4319
ExtraTreesRegressor,0.5457,0.5878,0.7387,0.1565,0.4488,0.4196,0.4586,0.5729,0.6772,0.2204,0.5984,0.4209
LinearRegression,5.0916,1.6532,2.2565,-6.8708,-0.0433,0.0163,2.4853,1.3465,1.5765,-3.2250,0.2458,0.0248
KNeighborsRegressor,0.6412,0.6220,0.8007,0.0089,0.3256,0.3644,0.5822,0.6814,0.7630,0.0102,0.3936,0.3283
SVR,0.6208,0.6227,0.7879,0.0403,0.3086,0.3853,0.5178,0.6335,0.7196,0.1197,0.4303,0.3989


In [28]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-5.354418349914161, -5.316879002745812, -6.56...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.131535090400979, -5.469807473519724, -5.3...","[-5.202178993453439, -5.232634570165066, -5.58...","[0.1242382138879358, 0.14373337516374318, 0.15..."
1,DecisionTreeRegressor,"[-4.73, -5.06, -6.85, -5.793469425, -6.9299170...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-6.25, -6.22, -6.17, -5.303671307, -6.569578...","[-5.4937051858, -6.072, -6.15, -5.7197026134, ...","[0.7361113112697358, 0.2060485379710323, 0.601..."
2,RandomForestRegressor,"[-5.48232826188, -5.391520198169998, -6.364656...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.497930698200004, -5.51785827859, -5.81246...","[-5.385590163172003, -5.476410728076001, -5.76...","[0.2006143404347011, 0.10319695746002353, 0.07..."
3,GradientBoostingRegressor,"[-5.667756811353129, -5.54820047151307, -6.851...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.447365744269249, -5.5911185425796, -6.008...","[-5.433312485239279, -5.4738550888872, -5.8895...","[0.39621646412331, 0.25091914429132384, 0.1917..."
4,AdaBoostRegressor,"[-5.30148539195238, -5.30148539195238, -6.1139...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.270455309699999, -5.418806133230769, -5.5...","[-5.256275101524894, -5.47217058383661, -5.522...","[0.1441962744596021, 0.29031927695059795, 0.22..."
5,XGBRegressor,"[-5.764666, -5.147919, -6.4477324, -5.8681383,...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.522504, -5.68638, -5.780648, -5.4122796, ...","[-5.2166243, -5.451589, -5.8832235, -5.617455,...","[0.4216381, 0.30115765, 0.30467716, 0.21026993..."
6,ExtraTreesRegressor,"[-5.373298738399999, -5.228926914429999, -6.07...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.235171998490006, -5.421000258870004, -5.7...","[-5.311246406306003, -5.562271749428002, -5.69...","[0.3257166882703574, 0.20303047231722762, 0.07..."
7,LinearRegression,"[-10.0, -5.74848892264249, -5.871378872182627,...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-8.89543216109687, -10.0, -3.9, -4.253783602...","[-9.187603680712055, -7.499291887549445, -5.67...","[0.8598541725240082, 2.055134308127762, 1.6447..."
8,KNeighborsRegressor,"[-5.613333333333333, -5.786666666666666, -6.07...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.359999999999999, -5.359999999999999, -5.6...","[-5.306666666666667, -5.526000000000001, -5.39...","[0.29388584329444783, 0.13724916514621652, 0.1..."
9,SVR,"[-5.411402691197683, -5.305429792103938, -5.86...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.105315830130418, -5.440219986114503, -5.6...","[-5.217731689524889, -5.38739966704816, -5.591...","[0.3181027661947282, 0.21389718024989055, 0.03..."
